# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116849e+02     1.509131e+00
 * time: 0.8115789890289307
     1     1.075531e+01     1.046395e+00
 * time: 3.2571301460266113
     2    -1.165680e+01     1.064526e+00
 * time: 3.9463560581207275
     3    -3.395038e+01     7.830277e-01
 * time: 4.987536191940308
     4    -4.747293e+01     5.369070e-01
 * time: 6.0273730754852295
     5    -5.697193e+01     2.256415e-01
 * time: 7.083360195159912
     6    -5.981305e+01     1.441000e-01
 * time: 7.777106046676636
     7    -6.091179e+01     5.638094e-02
 * time: 8.481749057769775
     8    -6.135949e+01     4.571594e-02
 * time: 9.17664098739624
     9    -6.161954e+01     4.079839e-02
 * time: 9.908788204193115
    10    -6.183557e+01     2.822450e-02
 * time: 10.62145709991455
    11    -6.199341e+01     2.122035e-02
 * time: 11.31079912185669
    12    -6.207476e+01     1.847499e-02
 * time: 11.986379146575928
    13    -6.212586e+01     1.567392e-02
 * time: 12.662317037582397
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671061
    AtomicLocal         -18.8557638
    AtomicNonlocal      14.8522619
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485361 
    Xc                  -19.3336817

    total               -62.261666458426
